# Infer CYTOSOL - part 3️⃣

 🚧 WIP 🚧 (🚨🚨🚨🚨 Steps 4-9 depend on establishing a good solution from part 2️⃣.)
--------------

## OBJECTIVE:  Infer sub-cellular component #3: CYTOSOL  in order to understand interactome 

To measure shape, position, and size of the cytosol of the cell body -- the soma minus the nucleus.    

Dependencies:
The CYTOSOL inference rely on the NUCLEI AND SOMA inference.  Therefore all of the sub-cellular objects rely on this segmentation.





# IMPORTS

In [1]:
# top level imports
from pathlib import Path
import os, sys
from collections import defaultdict

import numpy as np

from skimage.morphology import binary_erosion
# # package for io 
from aicsimageio import AICSImage

import napari

### import local python functions in ../infer_subc_2d
sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))

from infer_subc_2d.utils.file_io import (read_input_image, 
                                                                    list_image_files, 
                                                                    export_ome_tiff, 
                                                                    etree_to_dict, 
                                                                    save_parameters, 
                                                                    load_parameters, 
                                                                    export_ndarray)
from infer_subc_2d.utils.img import *
from infer_subc_2d.organelles.nuclei import infer_NUCLEI
from infer_subc_2d.organelles.soma import infer_SOMA

%load_ext autoreload
%autoreload 2

test_img_n = 5

# IMAGE PROCESSING Objective 3:  infer CYTOSOL

## summary of steps

➡️ INPUT
- labeled NUCLEI (objective #1)
- labeled SOMA (objective #2)

PRE-PROCESSING
- erode Nuclei (shrink)

CORE PROCESSING
  - XOR(NUCLEI, SOMA)

- POST-PROCESSING
  - N/A

OUTPUT ➡️ 
- mask of CYTOSOL
- CYTOSOL object (intensity)


------------------------
# LOAD RAW IMAGE DATA
Identify path to _raw_ image data and load our example image


In [2]:
# build the datapath
# all the imaging data goes here.
data_root_path = Path(os.path.expanduser("~")) / "Projects/Imaging/data"

# linearly unmixed ".czi" files are here
data_path = data_root_path / "raw"
im_type = ".czi"

# get the list of all files
img_file_list = list_image_files(data_path,im_type)
test_img_name = img_file_list[test_img_n]


In [4]:

bioim_image = read_input_image(test_img_name)
img_data = bioim_image.image
raw_meta_data = bioim_image.raw_meta
ome_types = []
meta_dict = bioim_image.meta

# get some top-level info about the RAW data
channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']


/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)


## Get default parameters, including  "optimal" Z

takes ~ 4 seconds to calculate

In [5]:
load_Z_from_params = False


if load_Z_from_params:

    default_params = load_parameters( test_img_name.split("/")[-1], data_root_path / "intermediate" )

    ch_to_agg = default_params["ch_to_agg"]
    nuc_ch = default_params['nuc_ch']
    optimal_Z = default_params["optimal_Z"] #find_optimal_Z(img_data, nuc_ch, ch_to_agg) 
else:
    ch_to_agg = (1,2,3,4,5,6)
    nuc_ch = 0
    optimal_Z = find_optimal_Z(img_data, nuc_ch, ch_to_agg) 

    default_params = defaultdict(str, **{
        #"intensity_norm_param" : [0.5, 15]
        "intensity_norm_param" : [0],
        "gaussian_smoothing_sigma" : 1.34,
        "gaussian_smoothing_truncate_range" : 3.0,
        "dot_2d_sigma" : 2,
        "dot_2d_sigma_extra" : 1,
        "dot_2d_cutoff" : 0.025,
        "min_area" : 10,
        "low_level_min_size" :  100,
        "median_filter_size" : 4,
        "ch_to_agg" : (1,2,3,4,5,6), # exclude residual
        "nuc_ch" : 0,
        "optimal_Z": optimal_Z,
    })
    save_parameters(default_params, test_img_name.split("/")[-1], data_root_path / "intermediate" )
# make sure we have removed Z
if len(scale)>2:
    scale = scale[1:]


img_2D = img_data[:,[optimal_Z],:,:].copy()


## get the inferred nuclei object

(takes < 1 sec)

In [6]:


raw_nuclei = img_2D[0].copy()
NU_object, NU_label, out_p =  infer_NUCLEI(raw_nuclei.copy(), default_params) 


intensity normalization: min-max normalization with NO absoluteintensity upper bound


## get the inferred soma object

(takes < 1 sec)

In [8]:
raw_soma = (4. * img_2D[1].copy() + 
                               1. * img_2D[5].copy() + 
                               1. * img_2D[7].copy() )

SO_object, SO_label, out_p =  infer_SOMA(raw_soma.copy(), NU_label, default_params) 


intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound
intensity normalization: min-max normalization with NO absoluteintensity upper bound


## PRE-PROCESSING

In [23]:
###################
# PRE_PROCESSING
###################

#NU_eroded1 = morphology.binary_erosion(NU_object,  footprint=morphology.ball(3) )
NU_eroded = binary_erosion(NU_object)  



## CORE PROCESSING

In [13]:

###################
# CORE_PROCESSING
###################

# logical  SO - NU_eroded
# CY_labels = np.zeros_like(SO_labels)
CY_object = np.logical_and(SO_object,~NU_eroded)
# CY_labels[CY_object] = 1

In [24]:



viewer = napari.view_image( #viewer.add_image(
    CY_object,
    scale=scale,
    opacity=0.3,
)    

viewer.add_image(
    NU_eroded,
    scale=scale,
    opacity=0.3,
)    

viewer.add_image(
    NU_object,
    scale=scale,
    opacity=0.3,
)    


<Image layer 'NU_object' at 0x17890d310>

In [15]:
CY_object.shape

(1, 768, 768)

# DEFINE `infer_CYTOSOL` function

Based on the _prototyping_ above define the function to infer cytosol.  


In [16]:
# copy this to base.py for easy import

def _infer_CYTOSOL(SO_object, NU_object, erode_NU = True):
    """
    Procedure to infer NUCLEI from linearly unmixed input.

    Parameters:
    ------------
    SO_object: np.ndarray
        a 3d image containing the NUCLEI signal

    NU_object: np.ndarray
        a 3d image containing the NUCLEI signal

    erode_NU: bool
        should we erode?

    Returns:
    -------------
    CY_object: np.ndarray (bool)
      
    """

    #NU_eroded1 = morphology.binary_erosion(NU_object,  footprint=morphology.ball(3) )
    if erode_NU:
        CY_object = np.logical_and(SO_object,~morphology.binary_erosion(NU_object) )
    else:
        CY_object = np.logical_and(SO_object,~NU_object)
    return CY_object


--------------------------

# TEST `infer_CYTOSOL` exported functions


##
`infer_CYTOSOL` procedure

In [18]:
from infer_subc_2d.organelles.cytosol import infer_CYTOSOL

CY_object1 =  infer_CYTOSOL(SO_label, NU_label) 

CY_object =  infer_CYTOSOL(SO_object, NU_object) 


In [22]:


viewer.scale_bar.visible = True
viewer.add_image(
    CY_object1,
    scale=scale 
)

viewer.add_labels(
    CY_object,
    scale=scale 
)

viewer.add_labels(
    NU_object,
    scale=scale 
)

viewer.add_image(
    NU_eroded,
    scale=scale 
)

<Image layer 'NU_eroded' at 0x183afcfd0>